In [1]:
import math
import numpy as np

In [2]:
pathToData = "../data/money/money.csv"
pathToNames = "../data/money/money.names"

In [3]:
# pathToData = "../data/iris/iris.data"
# pathToNames = "../data/iris/iris.names"

In [4]:
# def __init__(self, pathToData,pathToNames):
filePathToData = pathToData
filePathToNames = pathToNames
data = []
classes = []
numAttributes = -1 
attrValues = {}
attributes = []
tree = None

In [5]:
# def fetchData(self):
with open(filePathToNames, "r") as file: 
    classes = file.readline()
    classes = [x.strip() for x in classes.split(",")]
    #add attributes
    for line in file:
        [attribute, values] = [x.strip() for x in line.split(":")]
        values = [x.strip() for x in values.split(",")]
        attrValues[attribute] = values

In [6]:
numAttributes = len(attrValues.keys())
attributes = list(attrValues.keys())
with open(filePathToData, "r") as file:
    for line in file:
        row = [x.strip() for x in line.split(",")]
        if row != [] or row != [""]:
            data.append(row)

In [7]:
def isAttrDiscrete(attribute):
    if attribute not in attributes:
        raise ValueError("Attribute not listed")
    elif len(attrValues[attribute]) == 1 and attrValues[attribute][0] == "continuous":
        return False
    else:
        return True

In [8]:
# def preprocessData(self):
for index,row in enumerate(data):
    for attr_index in range(numAttributes):
        if(not isAttrDiscrete(attributes[attr_index])): # Jika data nya numerik, jadiin float
            data[index][attr_index] = float(data[index][attr_index])

In [9]:
def log(x):
    if x == 0:
        return 0
    else:
        return math.log(x,2)

In [10]:
def entropy(dataSet):
    S = len(dataSet)
    if S == 0:
        return 0
    num_classes = [0 for i in classes]
    for row in dataSet:
        classIndex = list(classes).index(row[-1])
        num_classes[classIndex] += 1
    num_classes = [x/S for x in num_classes]
    ent = 0
    for num in num_classes:
        ent += num*log(num)
    return ent*-1

In [11]:
def gain(unionSet, subsets):
    #input : data and disjoint subsets of it
    #output : information gain
    S = len(unionSet)
    #calculate impurity before split
    impurityBeforeSplit = entropy(unionSet)
    
    #calculate impurity after split
    weights = [len(subset)/S for subset in subsets]
    impurityAfterSplit = 0
    splitInfo = 0
#     for i in range(len(subsets)):
        
#         splitInfo -= weights[i] * log(weights[i])
#         print("weights[{i}] * log(weights[{i}]) : ({a}/{b})  * log({a}/{b})".format(i = i, 
#                                                     a = len(subsets[i]),
#                                                     b = S))
#         impurityAfterSplit += weights[i]*entropy(subsets[i])
#         print("weights[{i}]*entropy(subsets[{i}]): ({a}/{b}) * {entropy}".format(i = i, 
#                                                     weight = weights[i], 
#                                                     a = len(subsets[i]),
#                                                     b = S,
#                                                     entropy = entropy(subsets[i])))
#     print("\timpurityAfterSplit")
    for i in range(len(subsets)):
        impurityAfterSplit += weights[i]*entropy(subsets[i])
#         print("weights[{i}]*entropy(subsets[{i}]): ({a}/{b}) * {entropy} = {result}".format(i = i, 
#                                                     weight = weights[i], 
#                                                     a = len(subsets[i]),
#                                                     b = S,
#                                                     entropy = entropy(subsets[i]),
#                                                     result = weights[i]*entropy(subsets[i])))
#     print("\tsplitInfo")
    for i in range(len(subsets)):

        splitInfo -= weights[i] * log(weights[i]) 
#         print("weights[{i}] * log(weights[{i}]) : ({a}/{b})  * log({a}/{b}) = {result}".format(i = i, 
#                                                     a = len(subsets[i]),
#                                                     b = S,
#                                                     result = weights[i] * log(weights[i])))
        
    #calculate total gain
    totalGain = (impurityBeforeSplit - impurityAfterSplit)
    if splitInfo == 0:
        gainRatio = 0 
    else:
        gainRatio = (impurityBeforeSplit - impurityAfterSplit) / splitInfo
    
#     print("gainRatio = (impurityBeforeSplit - impurityAfterSplit) / splitInfo : \n {gainRatio} = ({impurityBeforeSplit} - {impurityAfterSplit}) / {splitInfo}".format(gainRatio = gainRatio, impurityBeforeSplit = impurityBeforeSplit, impurityAfterSplit = impurityAfterSplit, splitInfo = splitInfo))
#     print("totalGain : {}".format(totalGain))
    
    return gainRatio

In [12]:
def allSameClass(data):
    if len(data) == 0:
        return False
    for row in data:
        if row[-1] != data[0][-1]:
            return False
    return data[0][-1]

In [13]:
def getMajClass(curData):
    freq = [0]*len(classes)
    for row in curData:
        index = classes.index(row[-1])
        freq[index] += 1
    maxInd = freq.index(max(freq))
    return classes[maxInd]

In [14]:
class Node:
    def __init__(self, isLeaf, parent_label, label, split_on, threshold):
        self.isLeaf = isLeaf
        self.parent_label = parent_label
        self.label = label
        self.split_on = split_on
        self.threshold = threshold
        self.children = []

In [15]:
def splitAttribute(curData, curAttributes):
    splitted = []
    maxEnt = -1*float("inf")
    best_attribute = -1
    #None for discrete attributes, threshold value for continuous attributes
    best_threshold = None
    for attribute in curAttributes:
        indexOfAttribute = attributes.index(attribute)
#         print("\nAttribute : {}".format(attribute))
        if isAttrDiscrete(attribute):
            #split curData into n-subsets, where n is the number of 
            #different values of attribute i. Choose the attribute with
            #the max gain
            valuesForAttribute = attrValues[attribute]
            subsets = [[] for a in valuesForAttribute]
            for row in curData:
                for index in range(len(valuesForAttribute)):
                    if row[indexOfAttribute] == valuesForAttribute[index]:
                        subsets[index].append(row)
                        break
            e = gain(curData, subsets)
            if e > maxEnt:
                maxEnt = e
                splitted = subsets
                best_attribute = attribute
                best_threshold = None
        else:
            #sort the data according to the column.Then try all 
            #possible adjacent pairs. Choose the one that 
            #yields maximum gain
            curData.sort(key = lambda x: x[indexOfAttribute])
            for j in range(0, len(curData) - 1):
                if curData[j][indexOfAttribute] != curData[j+1][indexOfAttribute]:
                    threshold = (curData[j][indexOfAttribute] + curData[j+1][indexOfAttribute]) / 2
                    less = []
                    greater = []
                    for row in curData:
                        if(row[indexOfAttribute] > threshold):
                            greater.append(row)
                        else:
                            less.append(row)
#                     print("threshold: {}".format(threshold))
                    e = gain(curData, [less, greater])
                    if e >= maxEnt:
                        splitted = [less, greater]
                        maxEnt = e
                        best_attribute = attribute
                        best_threshold = threshold
    return (best_attribute,best_threshold,splitted)


In [36]:
def recursiveGenerateTree(curData, curAttributes, parent_label):
    allSame = allSameClass(curData)

    if len(curData) == 0:
        #Empty data 
        return Node(True, parent_label, parent_label, None, None)
    elif allSame is not False:
        #return a node with that class
        return Node(True, parent_label, allSame, None, None)
    elif len(curAttributes) == 0:
        #return a node with the majority class
        majClass = getMajClass(curData)
        return Node(True, parent_label, majClass, None, None)
    else:
        #Not a leaf
        (best_attribute,best_threshold,splitted) = splitAttribute(curData, curAttributes)
        remainingAttributes = curAttributes[:]
        remainingAttributes.remove(best_attribute)
        majClass = getMajClass(curData)
        
        node = Node(False, parent_label, majClass, best_attribute, best_threshold)
        node.children = [recursiveGenerateTree(subset, remainingAttributes, majClass) for subset in splitted
                        ]
        return node

In [37]:
def printNode(node, indent=""):
    if not node.isLeaf:
        if node.threshold is None:
            #discrete
            for index,child in enumerate(node.children):
                if child.isLeaf:
#                     if child.label != 'Empty':
                        print(indent + node.split_on + " = " + attrValues[node.split_on][index] + " : " + child.label)
                else:
                    print(indent + node.split_on + " = " + attrValues[node.split_on][index] + " : ")
                    printNode(child, indent + "	")
        else:
            #numerical
            leftChild = node.children[0]
            rightChild = node.children[1]
            if leftChild.isLeaf:
#                 if leftChild.label != 'Empty':
                    print(indent + node.split_on + " <= " + str(node.threshold) + " : " + leftChild.label)
            else:
                print(indent + node.split_on + " <= " + str(node.threshold)+" : ")
                printNode(leftChild, indent + "	")

            if rightChild.isLeaf:
#                 if rightChild.label != 'Empty':
                    print(indent + node.split_on + " > " + str(node.threshold) + " : " + rightChild.label)
            else:
                print(indent + node.split_on + " > " + str(node.threshold) + " : ")
                printNode(rightChild , indent + "	")


In [62]:
def predictOne(input, node):
    if node.isLeaf: 
#         print(node.label)
        return node.label
    else: 
        split_on = node.split_on
        input_value = input[attributes.index(split_on)]
        if node.threshold is None:
            #discrete
            child_index = attrValues[split_on].index(input_value)
            child_node = node.children[child_index]
#             print(split_on, child_index)
            return predictOne(input, child_node)
        else: 
            #numerical
            threshold_value = node.threshold
            if input_value <= threshold_value:
                child_node = node.children[0]
            else:
                child_node = node.children[1]
#             print(split_on, input_value, threshold_value)
            return predictOne(input, child_node)
            
def predict(inputs, node):
    return list(map(predictOne, inputs, [node] * len(inputs)))
        

In [63]:
def evaluate(data, node):
    prediction = np.array(predict(data, node))
    real_target = np.array(data)[:,-1]
    accuracy = sum(real_target == prediction) / len(data)
    return accuracy
    

In [64]:
tree = recursiveGenerateTree(curData = data, curAttributes = attributes, parent_label = None)

In [65]:
# def printTree(self):
printNode(tree)

hoursperweek <= 55.0 : 
	education <= 14.5 : 
		age <= 71.5 : 
			relationship = 0 : 
				race = 0 : 
					sex = 0 : No
					sex = 1 : No
				race = 1 : 
					sex = 0 : No
					sex = 1 : No
			relationship = 1 : 
				race = 0 : No
				race = 1 : 
					sex = 0 : No
					sex = 1 : No
			relationship = 2 : 
				race = 0 : No
				race = 1 : 
					sex = 0 : No
					sex = 1 : No
		age > 71.5 : Yes
	education > 14.5 : Yes
hoursperweek > 55.0 : Yes


In [72]:
evaluate(data, tree) # akurasi data training

0.75

In [ ]:
#coretan 

In [66]:
data[1],attributes

([50.0, 13.0, '0', '1', '1', 13.0, 'No'],
 ['age', 'education', 'relationship', 'race', 'sex', 'hoursperweek'])

In [67]:
predictOne(data[1],tree)

'No'

In [68]:
predict(data[:10], tree)

['No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No']

In [69]:
list(np.array(data[:10])[:,-1])

['No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No']

In [70]:
len(data)

60

In [71]:
evaluate(data, tree)

0.75

In [54]:
np.array([1,2]) == np.array([1,3])

array([ True, False])

In [52]:
['No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No'] == ['No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No']

False

In [50]:
np.array(data[:10])[:,-1].shape

(10,)

In [ ]:
data[:10], attributes

In [ ]:
new_input = [67.0, 6.0, '0', '0', '0', 50.0]

In [ ]:
predictOne(new_input, tree)

In [ ]:
tree.parent_label

In [ ]:
tree.label

In [ ]:
tree.children[0].parent_label

In [ ]:
tree.children[0].label

In [ ]:
tree.children[1].parent_label

In [ ]:
tree.children[1].label